# PRICING A PHOENIX CALL ON NVIDIA

TO MODIFY THE SIZE OF THE COUPON WE NEED TO CHANGE I2 WICH IS EQUAL TO 1 BY
DEFAULT THAT REPRESENT A 10 % COUPON SIZE

In [ ]:
def payoff(t_steps, TtM, Drift, Vol, Disc, S_0, S_k, S_p, N, I, RND):

    # INPUT:
    # t_steps : time steps
    # TtM     : time to maturity
    # Drift   : drift list by time
    # Vol     : volatility list by time
    # Disc    : discount rate
    # S_0     : underlying initial value
    # S_k     : kickout barrier
    # S_p     : protection barrier
    # N       : nominal value
    # I       : yearly interest over the nominal
    # RND     : random terms


    from numpy import exp, sqrt

    # vars
    S_prev = S_0
    TtM[TtM.index[0]-1] = 0 # solve index issue in 'dt'

    # simu
    for t in t_steps:
        # diff = vol[t] * random() # check it # diffusion term

        # underlying dynamics
        dt = TtM[t] - TtM[t-1]
        S_t = S_prev * exp((Drift[t] - 0.5 * Vol[t] ** 2) * dt + Vol[t] * RND[t] * sqrt(dt))
        # update previous value
        S_prev = S_t

        # kick out barrier touched at t
        if S_t >= S_k:
            return (1 + TtM[t] * I) * exp(- Disc[t] * TtM[t])

    # kick out barrier never touched before the maturity
    if S_t > S_p:
        return (exp(- Disc[t] * TtM[t]))
    else:
        return (S_t / S_0 * exp(- Disc[t] * TtM[t]))



def monteCarloPrice(t_steps, TtM, Drift, Vol, Disc, S_0, S_k, S_p, N, I, n_simu, RND):

    # INPUT:
    # t_steps : time steps
    # TtM     : time to maturity
    # Drift   : drift list by time
    # Vol     : volatility list by time
    # Disc    : discount rate
    # S_0     : underlying initial value
    # S_k     : kickout barrier
    # S_p     : protection barrier
    # N       : nominal value
    # I       : yearly interest over the nominal
    # n_simu  : number of simulations
    # RND     : random terms

    # OUTPUT:
    # out     : autocallable structure price

    import numpy as np
    import pandas as pd
    import time

    if RND == None:
        # generate pseudo-random sequence
        RND = pd.DataFrame(np.random.randn(int(n_simu), len(t_steps)), columns=t_steps)

    # starting_t = time.time()

    payoffs = [0] * int(n_simu) # initializes list
    for i in range(int(n_simu)):
        payoffs[i] = payoff(t_steps, TtM, Drift, Vol, Disc, S_0, S_k, S_p, N, I, RND.iloc[i])

    # elapsed_t = time.time() - starting_t
    # print('\nMonte Carlo simulation completed in', elapsed_t, 's')

    return sum(payoffs) / n_simu

In [ ]:
import ipywidgets as widgets
from IPython.display import display

# Assuming monteCarloPrice function is defined as in the provided code.

def monte_carlo_interface(monteCarloPrice_func):
    # Input widgets
    t_steps_widget = widgets.IntSlider(min=1, max=100, step=1, value=10, description='Time Steps:')
    ttm_widget = widgets.FloatSlider(min=0.1, max=10, step=0.1, value=1, description='Time to Maturity:')
    drift_widget = widgets.FloatText(value=0.05, description='Drift:')
    vol_widget = widgets.FloatText(value=0.2, description='Volatility:')
    disc_widget = widgets.FloatText(value=0.01, description='Discount Rate:')
    s0_widget = widgets.FloatText(value=100, description='Initial Price:')
    sk_widget = widgets.FloatText(value=110, description='Kickout Barrier:')
    sp_widget = widgets.FloatText(value=90, description='Protection Barrier:')
    n_widget = widgets.IntText(value=10000, description='Nominal Value:')
    i_widget = widgets.FloatText(value=0.02, description='Yearly Interest:')
    n_simu_widget = widgets.IntText(value=1000, description='Number of Simulations:')

    # Output widget
    output_widget = widgets.Output()

    #Button to run
    run_button = widgets.Button(description="Run Simulation")


    def run_simulation(b):
        with output_widget:
            output_widget.clear_output()
            try:
                price = monteCarloPrice_func(
                    t_steps=list(range(1, t_steps_widget.value+1)),
                    TtM=pd.Series(data=[ttm_widget.value * i / t_steps_widget.value for i in range(t_steps_widget.value + 1)], index=list(range(t_steps_widget.value + 1))),
                    Drift=pd.Series(data=[drift_widget.value for i in range(t_steps_widget.value + 1)], index=list(range(t_steps_widget.value + 1))),
                    Vol=pd.Series(data=[vol_widget.value for i in range(t_steps_widget.value + 1)], index=list(range(t_steps_widget.value + 1))),
                    Disc=pd.Series(data=[disc_widget.value for i in range(t_steps_widget.value + 1)], index=list(range(t_steps_widget.value + 1))),
                    S_0=s0_widget.value,
                    S_k=sk_widget.value,
                    S_p=sp_widget.value,
                    N=n_widget.value,
                    I=i_widget.value,
                    n_simu=n_simu_widget.value,
                    RND=None # No pre-generated random numbers are passed to function
                )

                print(f"The calculated price is: {price*1000}")
            except Exception as e:
                print(f"An error occurred: {e}")

    run_button.on_click(run_simulation)


    # Display the interface
    display(t_steps_widget, ttm_widget, drift_widget, vol_widget, disc_widget, s0_widget, sk_widget, sp_widget, n_widget, i_widget, n_simu_widget, run_button, output_widget)

# Example usage:
monte_carlo_interface(monteCarloPrice)

IntSlider(value=10, description='Time Steps:', min=1)

FloatSlider(value=1.0, description='Time to Maturity:', max=10.0, min=0.1)

FloatText(value=0.05, description='Drift:')

FloatText(value=0.2, description='Volatility:')

FloatText(value=0.01, description='Discount Rate:')

...

IntText(value=10000, description='Nominal Value:')

FloatText(value=0.02, description='Yearly Interest:')

IntText(value=1000, description='Number of Simulations:')

Button(description='Run Simulation', style=ButtonStyle())

Output()